# Importing the libraries and loading the dataset

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
df = pd.read_csv('2024_Accidentalidad.csv', sep=';')

In [24]:
df.sample(5)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
36433,2024S028044,06/10/2024,22:00:00,AVDA. PABLO NERUDA / AVDA. ALBUFERA,16,13,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Pasajero,De 55 a 59 años,Hombre,14.0,Sin asistencia sanitaria,445377.0,4471150.0,N,NaN
12001,2024S012085,02/04/2024,7:03:00,CALL. SAAVEDRA FAJARDO / GTA. PUENTE DE SEGOVIA,20,10,LATINA,Colisión fronto-lateral,NaN,Turismo,Conductor,De 35 a 39 años,Hombre,NaN,NaN,438491.0,4473956.0,N,NaN
24220,2024S020695,25/06/2024,18:15:00,"CALL. MARQUES DEL RISCAL, 6",6,7,CHAMBERÍ,Alcance,Despejado,Turismo,Conductor,De 65 a 69 años,Mujer,NaN,NaN,441282.0,4475771.0,N,NaN
8140,2024S007690,02/03/2024,13:05:00,AVDA. NUESTRA SEÑORA DE FATIMA / CALL. EUGENIA...,104,11,CARABANCHEL,Alcance,Lluvia débil,Turismo,Pasajero,De 50 a 54 años,Mujer,14.0,Sin asistencia sanitaria,436447.0,4469994.0,N,NaN
24291,2024S020736,26/06/2024,19:16:00,CALL. CORAZON DE MARIA / CALL. CARDENAL SILICEO,21,5,CHAMARTÍN,Alcance,Se desconoce,Turismo,Conductor,De 30 a 34 años,Mujer,14.0,Sin asistencia sanitaria,443299.0,4477062.0,N,NaN


In [25]:
df.shape

(40165, 19)

# Tarea 1
*Realiza un análisis descriptivo del dataset. Analiza la distribución de los datos
por cada una de las columnas, realiza los pasos de pre-procesamiento necesarios,
justificando adecuadamente las acciones tomadas. Se deberá hacer uso de gráficas para
entender los datos y las decisiones adoptadas.*

## 1. Preprocess

### 1.1. NaN

In [26]:
def null_percentage(df, percentage):
    null_ratio = {}
    for col in df.columns:
        ratio = df[col].isna().sum() / len(df) * 100
        if ratio > percentage:
            null_ratio[col] = ratio
           
    return null_ratio

In [76]:
null_percentage(df, 0)

{'tipo_accidente': 0.0024903498941601294,
 'estado_meteorológico': 11.595069107209563,
 'tipo_vehiculo': 0.7994023160254016,
 'positiva_alcohol': 0.3436682853940979}

According to Adrian: 
- everything that is lower than 10 -> we drop the rows
- estado meteorológico -> try to inpute the data

accidente, coordenadas,  -> eliminar filas
tipo vehiculo -> cambiar NaN por 'Sin especificar'
estado mete -> input

#### 1.1.1. Cooridinates

As coordinates and adress give the same information, we decided that it makes more sense to only keep the coordinates, because they are more specific, preserve the information about the distance and are numeric values.

We will drop *localizacion* and *numero* columns. We decided to keep the *distrito* column, as it gives more iformation than just an adress.

We can see that the percentage of NaNs is excacly the same for *x* and *y* coordinates. Anyway, we will check if it is always true that lack of one of the coordinates implies the lack of the other.

In [66]:
df[(df['coordenada_x_utm'].isna() == True) & (df['coordenada_y_utm'].isna() == True)]

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol
34485,2024S026886,24/09/2024,2:05:00,PASEO. CASTELLANA / CALL. HERMANOS PINZON,124,5,Alcance,Despejado,Turismo,Conductor,De 35 a 39 años,Hombre,14.0,NaN,NaN,N
34486,2024S026886,24/09/2024,2:05:00,PASEO. CASTELLANA / CALL. HERMANOS PINZON,124,5,Alcance,Despejado,Turismo,Pasajero,De 40 a 44 años,Mujer,6.0,NaN,NaN,N
39263,2024S029749,25/10/2024,23:30:00,"PASEO. RECOLETOS, 21",21,1,Caída,Despejado,VMU eléctrico,Conductor,De 50 a 54 años,Hombre,3.0,NaN,NaN,N
39388,2024S029842,27/10/2024,1:40:00,"CALL. EMBAJADORES, CAJA MAGICA",480,13,Colisión frontal,Despejado,Turismo,Conductor,De 21 a 24 años,Hombre,14.0,NaN,NaN,N
39389,2024S029842,27/10/2024,1:40:00,"CALL. EMBAJADORES, CAJA MAGICA",480,13,Colisión frontal,Despejado,Turismo,Pasajero,De 45 a 49 años,Mujer,2.0,NaN,NaN,N
39390,2024S029842,27/10/2024,1:40:00,"CALL. EMBAJADORES, CAJA MAGICA",480,13,Colisión frontal,Despejado,Turismo,Pasajero,De 50 a 54 años,Hombre,2.0,NaN,NaN,N
39391,2024S029842,27/10/2024,1:40:00,"CALL. EMBAJADORES, CAJA MAGICA",480,13,Colisión frontal,Despejado,Turismo,Pasajero,De 50 a 54 años,Mujer,2.0,NaN,NaN,N
39531,2024S029936,28/10/2024,9:15:00,CALL. DIVINO PASTOR / CALL. RUIZ,23,1,Atropello a animal,Despejado,Turismo,Conductor,Desconocido,Desconocido,0.0,NaN,NaN,N
39767,2024S030072,29/10/2024,21:20:00,"CALL. GASOMETRO, 8A",8A,2,NaN,NaN,Motocicleta > 125cc,Conductor,De 18 a 20 años,Hombre,7.0,NaN,NaN,N
39768,2024S030072,29/10/2024,21:20:00,"CALL. GASOMETRO, 8A",8A,2,NaN,NaN,Turismo,Conductor,De 45 a 49 años,Hombre,0.0,NaN,NaN,N


It looks like our hipotesis is right, but we will also check the numbers.

In [69]:
len_1 = len(df[(df['coordenada_x_utm'].isna() == True) & (df['coordenada_y_utm'].isna() == True)])
len_1

10

In [71]:
len_2 = 0.024897298643097227/100 * len(df)
len_2

10.0

Everything aligns, so now we can drop the rows that are missing the coordinates. There are just a few, so it will not affect our analysis.

In [ ]:
df = df.dropna(subset=['coordenada_x_utm', 'coordenada_y_utm'])

#### 1.1.2. Drugs

Column containg data about drug influence seems to be very important, especially when we are analysing the car accidents. However, it contains more than 99.5% of NaN values, so it does not give us any information. Inputing the data would make the column completely artificial. 

In [54]:
df['positiva_droga'].value_counts()

positiva_droga
1.0    170
Name: count, dtype: int64

We can see that there are only 170 positive values in the column, whereas the others are NaNs.

We will drop the column for now, but in later analysis we will see if inputing the data will improve the results.

In [56]:
df.drop(columns=['positiva_droga'], inplace=True)

#### 1.1.3. Vehicle -> para pegar

#### 1.1.4. Accident type -> para pegar

#### 1.1.5. Alcohol

At frist, we will see how many missing values we have.

In [86]:
len(df[df['positiva_alcohol'].isna() == True])

138

As we know in one accident there are more than just one car involved. We will check if the missing data belongs to only a few accidents or if each of them has a different *num_expediente* value. The latter will obviously make things more complicated.

In [87]:
len(df['num_expediente'][df['positiva_alcohol'].isna() == True].unique())

138

It looks, like there is as many accidents as missing values. We cannot reaaly drop those rows, because they are instances of different accident. If we delete the row containng data about one of the crushed cars, we are also loosing data for another participants of the accident.

We can check if the type of person is always a vehicle conductor. It could appear that the person is a child or a passanger. In this case the value of *positiva_alcohol* would not affect the data.

In [96]:
# print the rows where the person is not a driver and alhocol is nan
df[(df['tipo_persona'] != 'Conductor') & (df['positiva_alcohol'].isna() == True)]

,num_expediente,fecha,hora,cod_distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol


Unfortunatelly, it seems that the only people we are lacking information for are the drivers. That will not help us. 

After analysing the case, we decided to ytake a risk and have 138 rows with inputed data. That may results in bad classification, but we do not have other choice. As stated earlier, we cannot drop the rows, as they are a "part" of a whole another accident. 


**Decission:** Finally, we decided to treat empty values as False ('N'), meaning no influence of alcohol.

In [97]:
df['positiva_alcohol'].fillna('N', inplace=True)

C:\Users\48726\AppData\Local\Temp\ipykernel_2352\2019537947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['positiva_alcohol'].fillna('N', inplace=True)


### 1.2. Redundant data

There are two pairs of columns that contain exactly the same information: *distrito* and *cod_distrito* and *lesividad* and *cod_lesividad*.

We decided to drop the columns containg names, as the others are already "encoded". However, first we will create dictionaries that will facilitate the interpretation of the data. We will also change the codes to numeric values, so that we can use them in later analysis.

#### 1.2.1. Distrito

At first, we have to see if the codes align with the names.

In [28]:
df['distrito'].value_counts()

distrito
PUENTE DE VALLECAS     3253
CHAMARTÍN              2864
CARABANCHEL            2862
SALAMANCA              2636
CIUDAD LINEAL          2448
CENTRO                 2254
ARGANZUELA             2175
FUENCARRAL-EL PARDO    2032
RETIRO                 2013
MONCLOA-ARAVACA        1990
LATINA                 1934
TETUÁN                 1894
SAN BLAS-CANILLEJAS    1884
CHAMBERÍ               1788
USERA                  1718
HORTALEZA              1473
VILLAVERDE             1426
MORATALAZ              1093
VILLA DE VALLECAS      1046
BARAJAS                 711
VICÁLVARO               671
Name: count, dtype: int64

In [29]:
df['cod_distrito'].value_counts()

cod_distrito
13    3253
5     2864
11    2862
4     2636
15    2448
1     2254
2     2175
8     2032
3     2013
9     1990
10    1934
6     1894
20    1884
7     1788
12    1718
16    1473
17    1426
14    1093
18    1046
21     711
19     671
Name: count, dtype: int64

We can see that the codes in the *cod_distrito* column are assigned taking into account the city zone, so the distance is preserved. 

Now, we create the dictionary contaning district names and codes.

In [30]:
district_keys = df['cod_distrito'].unique()    
district_names = df['distrito'].unique()

district_dict = dict(zip(district_keys, district_names))


In [31]:
district_dict

{16: 'HORTALEZA',
 1: 'CENTRO',
 7: 'CHAMBERÍ',
 6: 'TETUÁN',
 10: 'LATINA',
 9: 'MONCLOA-ARAVACA',
 3: 'RETIRO',
 13: 'PUENTE DE VALLECAS',
 14: 'MORATALAZ',
 11: 'CARABANCHEL',
 4: 'SALAMANCA',
 5: 'CHAMARTÍN',
 19: 'VICÁLVARO',
 2: 'ARGANZUELA',
 8: 'FUENCARRAL-EL PARDO',
 12: 'USERA',
 18: 'VILLA DE VALLECAS',
 21: 'BARAJAS',
 17: 'VILLAVERDE',
 20: 'SAN BLAS-CANILLEJAS',
 15: 'CIUDAD LINEAL'}

Now, we drop the *distrito* column.

In [32]:
df.drop(['distrito'], axis=1, inplace=True)

#### 1.2.2. Lesividad

We check, if the values align.

In [33]:
df['cod_lesividad'].value_counts()

cod_lesividad
14.0    13109
7.0      4720
2.0      1494
1.0      1111
6.0       950
3.0       475
5.0       453
4.0        24
Name: count, dtype: int64

In [34]:
df['lesividad'].value_counts()

lesividad
Sin asistencia sanitaria                                     13109
Asistencia sanitaria sólo en el lugar del accidente           4720
Ingreso inferior o igual a 24 horas                           1494
Atención en urgencias sin posterior ingreso                   1111
Asistencia sanitaria inmediata en centro de salud o mutua      950
Ingreso superior a 24 horas                                    475
Asistencia sanitaria ambulatoria con posterioridad             453
Fallecido 24 horas                                              24
Name: count, dtype: int64

In [35]:
df['lesividad'].unique(), df['cod_lesividad'].unique()

(array(['Ingreso inferior o igual a 24 horas', 'Sin asistencia sanitaria',
        'Asistencia sanitaria sólo en el lugar del accidente', nan,
        'Atención en urgencias sin posterior ingreso',
        'Ingreso superior a 24 horas',
        'Asistencia sanitaria inmediata en centro de salud o mutua',
        'Asistencia sanitaria ambulatoria con posterioridad',
        'Fallecido 24 horas'], dtype=object),
 array([ 2., 14.,  7., nan,  1.,  3.,  6.,  5.,  4.]))

As we know, those columns containg more than 40% of NaN values.

We could treat them as another category and encode them as well, naming it *'Desconocido'*. Inputing 40% of data uing other 60% does not really make sense. Nor does dropping almost half of the rows. So we decided to use encoding, as it is the most reasonbale option.

At first, we use *.fillna()* on those two columns, so that we can use the new values in our dictionary.

In [95]:
df['cod_lesividad'].fillna(0, inplace=True)
df['lesividad'].fillna('Sin especificar', inplace=True)

C:\Users\48726\AppData\Local\Temp\ipykernel_2352\3214936209.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['cod_lesividad'].fillna(0, inplace=True)
C:\Users\48726\AppData\Local\Temp\ipykernel_2352\3214936209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cod_lesividad'].fillna(0, inplace=True)


KeyError: 'lesividad'

In [47]:
lesividad_keys = df['cod_lesividad'].unique()    
lesividad_names = df['lesividad'].unique()

lesividad_dict = dict(zip(lesividad_keys, lesividad_names))


We check the results.

In [45]:
lesividad_dict

{2.0: 'Ingreso inferior o igual a 24 horas',
 14.0: 'Sin asistencia sanitaria',
 7.0: 'Asistencia sanitaria sólo en el lugar del accidente',
 0.0: 'Desconocido',
 1.0: 'Atención en urgencias sin posterior ingreso',
 3.0: 'Ingreso superior a 24 horas',
 6.0: 'Asistencia sanitaria inmediata en centro de salud o mutua',
 5.0: 'Asistencia sanitaria ambulatoria con posterioridad',
 4.0: 'Fallecido 24 horas'}

Now, we can drop the *lesividad* column.

In [49]:
df.drop(['lesividad'], axis=1, inplace=True)

In [50]:
df.sample(5)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
16171,2024S015132,30/04/2024,18:15:00,"CALL. SAN BERNARDO, 42",42,1,Alcance,NaN,Motocicleta > 125cc,Conductor,De 65 a 69 años,Hombre,0.0,439993.0,4475076.0,N,NaN
16241,2024S015170,01/05/2024,1:40:00,"CALL. ARCOS DE JALON, 70",70,20,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,Desconocido,Desconocido,0.0,447669.0,4474817.0,N,NaN
34463,2024S026872,23/09/2024,19:35:00,CALL. SERRANO / AVDA. CONCHA ESPINA,199,5,Colisión fronto-lateral,Nublado,Turismo,Pasajero,De 55 a 59 años,Mujer,14.0,442226.0,4478116.0,N,NaN
22534,2024S019651,13/06/2024,18:28:00,CALL. CALESAS / CALL. MIRASIERRA,13,12,Choque contra obstáculo fijo,NaN,Furgoneta,Conductor,Desconocido,Desconocido,0.0,440126.0,4471254.0,N,NaN
23627,2024S020337,21/06/2024,14:35:00,CALL. RAIMUNDO FERNANDEZ VILLAVERDE / CALL. MO...,15,6,Colisión lateral,Despejado,Turismo,Conductor,Desconocido,Desconocido,0.0,441003.0,4477529.0,N,NaN


In [51]:
df.shape

(40165, 17)

#### 1.2.3. Localización

As stated earlier, *localizacion* and *numero* columns are redundant, so we will drop them directly.

In [74]:
df.drop(['localizacion', 'numero'], axis=1, inplace=True)

C:\Users\48726\AppData\Local\Temp\ipykernel_2352\2735048898.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['localizacion', 'numero'], axis=1, inplace=True)


In [75]:
df.shape

(40155, 14)

## 1.3. Datetime columns -> para pegar

## 1.4. Encoding -> para pegar??

## otro

We want to check if every ID (*num_expediente* column) corresponds to only one row.

In [52]:
len(df['num_expediente'].unique())

16935

It is obviously false, as there are usually one there more vehicule/person involved in the accident.